In [1]:
import pandas as pd
import numpy as np
import os
from warnings import simplefilter
simplefilter("ignore")

In [4]:
def clean_data(df):
    df['Local Area Unemployment Statistics'] = df['Local Area Unemployment Statistics'].fillna('None')
    df = df.set_index('Local Area Unemployment Statistics', drop=True)
    df = df.loc[['State/Region/Division:','Year',2019,2020,2021,2022], :]
    df.columns = df.loc['Year']
    df.columns.name = 'month'
    state = df.loc['State/Region/Division:'][0]
    df = df.drop(['State/Region/Division:', 'Year'], axis=0)
    df.index.name = 'year'
    df = df.stack().reset_index(drop=False)
    df.index = pd.date_range(start="2019-01", end="2022-12", periods=None, freq="MS")
    df.index.name = 'Month'
    df['state'] = state
    df = df.rename(columns={0: 'unemployment_rate'}).drop(['year', 'month'], axis=1)
    df['unemployment_rate'] = df['unemployment_rate'].astype(np.float64)
    
    return df

In [15]:
unmp_list = []
for filename in os.listdir('./rawdata_states'):
    state_unmp = pd.read_excel(f'./rawdata_states/{filename}',
                            engine='openpyxl'
                           )

    unmp_list.append(clean_data(state_unmp))

pd.concat(unmp_list,axis=0).to_csv('./files/state_unemployment.csv', header=True, index=True)